# Python Stats: Hypothesis testing - Proportion test

## Cases of Proportion Test

In [1]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats.distributions as dist

import warnings
warnings.filterwarnings('ignore')

# set maximum columns display as 50
pd.set_option('display.max_columns',50) 

%matplotlib inline

In [2]:
# import the dataset
df = pd.read_csv('Salaries.csv', )
df.head()

# this dataset shows the salary information for workers in SF

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.00,400184.25,NaN,567595.43,567595.43,2011,NaN,San Francisco,NaN
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,NaN,538909.28,538909.28,2011,NaN,San Francisco,NaN
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.60,NaN,335279.91,335279.91,2011,NaN,San Francisco,NaN
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.00,56120.71,198306.90,NaN,332343.61,332343.61,2011,NaN,San Francisco,NaN
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.60,9737.00,182234.59,NaN,326373.19,326373.19,2011,NaN,San Francisco,NaN


In [3]:
df.shape

(148654, 13)

In [4]:
df.isnull().sum()
# no missing values in total pay and year

Id                       0
EmployeeName             0
JobTitle                 0
BasePay                609
OvertimePay              4
OtherPay                 4
Benefits             36163
TotalPay                 0
TotalPayBenefits         0
Year                     0
Notes               148654
Agency                   0
Status              148654
dtype: int64

In [5]:
df.drop(['Notes','Status'], axis = 1, inplace = True)
df.shape

(148654, 11)

**Notes:**
- in actual data analysis, we need to examine missing values and find out potential reasons for missing.
- here, we focus on showing the process of hypothesis testing.

## 1. hypothesis testing for one-proportion

**Example Scenario:**

- Assume we know that in 2011, 25% of employees nationwide (in U.S.) had a total pay of $100,000 and above.

- Here, we want to know:
  - whether the proportion of employees with total pay of $100,000 and above in SF is different from the nationwide proportion in the same year (2011). 


**Assumptions Checking:**
- Assume data were randomly sampled from all employees in SF 
- Sample size here is already big enough.

Here, we will run a **two-sided one-sample proportion test**:

**Null Hypothesis (H0): p1 = p0**

(Meaning: the proportion of employees with total pay of $100,000 and above in SF *is the same as* the nationwide proportion in the same year (2011))

**Alternative Hypothesis (H1): p1 != p0** 

(Meaning: the proportion of employees with total pay of $100,000 and above in SF is different from the nationwide proportion in the same year (2011))
   
Set alpha at 0.05

In [6]:
# calculate the proportion of employees with total pay of $100,000 and above in SF in year 2011

df_pay = df['TotalPay'][df['Year']== 2011]
print(df_pay.shape)

df_high_pay = df['TotalPay'][(df['TotalPay']>=100000) & (df['Year'] == 2011)]
print(df_high_pay.shape)

# 9524 employees out of 36159 employees sampled had a total pay of 100,000 and above

(36159,)
(9524,)


In [7]:
# calculate p1
p1 = df_high_pay.shape[0]/df_pay.shape[0]
p1

0.26339223982964133

### 1). mannual calculation

- z = (p0-p1)/se
- se = sqrt(p0*(1-p0)/n) or sqrt(p1*(1-p1)/n) : standard error of estimate

In [8]:
# calculate standard error of estimate
import numpy as np
p0 = 0.25
n = df_pay.shape[0]
se = np.sqrt(p1*(1-p1)/n)
se


0.002316388643197871

In [9]:
# # calculate standard error of estimate
# import numpy as np
# p0 = 0.25 # null proportion
# n = df_pay.shape[0]
# se = np.sqrt(p0*(1-p0)/n)
# se

In [10]:
# calculate z score
p0 = 0.25
p1 = 0.2634
z = (p1-p0)/se
z

5.7848669045021595

In [11]:
# # calculate p-value
pvalue = 2 * dist.norm.cdf(-np.abs(z)) # cdf: cumulative distribution function 
print(z, pvalue)
# p < 0.05, reject null hypothesis

5.7848669045021595 7.256976444300643e-09


In [12]:
# or use below to find p-value
import scipy
pvalue = scipy.stats.norm.pdf(abs(z))*2     # pdf: probability density function; here for normal distribution
pvalue 

# result should be close to the above approach 

4.3169400948403186e-08

In [13]:
# find Z values for the left and right tail when significane level - alpha = 0.05
print(st.norm.ppf(.025))
print(st.norm.ppf(.975))

# here, we can also see that the Z score (95%) = 3.92 < z = 4.086: should reject null hypothesis

NameError: name 'st' is not defined

### 2). built-in stats models

- for one sample proportion:

sm.stats.proportions_ztest(count, nobs, value)

- count: the number of observed successes in nobs trials; here, number of employee with total pay of 100,000 and above in SF in 2011
- nobs: total number of observations: total number of employees in SF in 2011 (sampled)
- value: for one-sample proportion = p0; for two sample proportion = prop[0]-prob[1]

**Attention:**

- in Python built-in statsmodels: when calculate standard error (se), the sample proportion (p1) is used
- however, the results should be very similar to using null proportion proportion (p0)


In [14]:
# use built in stats models to calculate z score and p-value
stat1, pvalue1 = sm.stats.proportions_ztest(df_high_pay.shape[0], 
                                            df_pay.shape[0], 0.25, alternative='two-sided')
stat1, pvalue1
# Normal approximation with estimated proportion in se

(5.7815167886304195, 7.403009152237138e-09)

In [15]:
# Exact binomial distribution  p-value  -- only provide p-value
pvalue2 = sm.stats.binom_test(df_high_pay.shape[0], 
                            df_pay.shape[0], 0.25, alternative='two-sided') # Exact binomial p-value
pvalue2

4.967808616616095e-09

## 2. hypothesis testing for differences in two proportions

- Let's assume we want to know whether the proportion of emplyees whose total pay is at least $100,000 in San
Francisco from year 2011 is different from that of 2012


Here, I will run **a two-sided two-sample proportion test**

**Null Hypothesis (H0): p2 - p1 = 0**

    - (Meaning: The proportion of employees with total pay of $100,000 and above in SF in 2011 *is the same as* that of 2012 )
 
**Alternative Hypothesis (H1): p2 - p1 !=0**

    - (Meaning: The proportion of employees with total pay of $100,000 and above in SF is different from that of 2012)
 
Set alpha at 0.05

### 1) mannual calculation

- z = ((p2-p1) - 0)/se
- standard error of estimate: se = sqrt{(p^*(1 - p^) ) * (1/n1 + 1/n2)}  
- and p^ = total number of sampled employees with total pay of $100,000 and above in 2011 and 2012 / total sampled employees in this 2011 and 2012

In [16]:
# from previous calculation:

p1 = df_high_pay.shape[0]/df_pay.shape[0]
p1

0.26339223982964133

In [17]:
# calculate proportion of highly paid employees in 2012
df_pay12 = df['TotalPay'][df['Year'] == 2012]
df_high_pay12 = df['TotalPay'][(df['TotalPay']>=100000) & (df['Year'] == 2012)]
p2 = df_high_pay12.shape[0]/df_pay12.shape[0]
p2

0.2720176249796007

In [18]:
# calculate p_hat
p_hat = (df_high_pay12.shape[0] + df_high_pay.shape[0])/ (df_pay.shape[0] + df_pay12.shape[0])
print(p_hat)

# calculate standard error of estimate
se = np.sqrt((p_hat*(1 - p_hat)) * (1/df_pay.shape[0] + 1/df_pay12.shape[0]))
print(se)

0.26774082961947204
0.0032794161706268003


In [19]:
# calculate Z score
z = ((p2-p1) - 0)/se
z

2.6301587542366662

In [20]:
# find pvalue
pvalue = 2 * dist.norm.cdf(-np.abs(z)) # cdf: cumulative distribution function 
print(z, pvalue)

# p < 0.05 , reject null

2.6301587542366662 0.008534500343848827


### 2) built-in stats models

In [21]:
# find pvalue
count = np.array([df_high_pay.shape[0],df_high_pay12.shape[0]]) 
nobs = np.array([df_pay.shape[0], df_pay12.shape[0]])
stat, pvalue = sm.stats.proportions_ztest(count, nobs, alternative='two-sided')
stat, pvalue

# same value as above, but using formula: ((p1-p2) -0) /se

(-2.6301587542366662, 0.008534500343848827)